In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import DeepFool
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics
from warnings import simplefilter

import joblib

2024-02-26 11:05:23.764349: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 11:05:23.767015: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 11:05:23.796539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 11:05:23.796597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 11:05:23.797614: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
#with two dataset splitted
dftrain = pd.read_csv("/home/jovyan/MQTTset/train70_reduced.csv") 
dftest = pd.read_csv("/home/jovyan/MQTTset/test30_reduced.csv")

# dftrain = pd.read_csv("train70.csv", low_memory=False) 
# dftest = pd.read_csv("test30.csv", low_memory=False)

simplefilter(action='ignore', category=FutureWarning)
seed = 7

#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']

print("Ready to generate train and test datasets")

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train = np.copy(x_train_scaled)
x_test = np.copy(x_test_scaled)

Ready to generate train and test datasets


In [4]:
model = joblib.load("/home/jovyan/MQTTset/dl/new_dl/dnn.joblib")

2024-02-26 11:05:29.623941: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://1cec13c142d04b8cb798d5230a461acf: INVALID_ARGUMENT: ram://1cec13c142d04b8cb798d5230a461acf is a directory.


In [5]:
x_test_trimmed = x_test[1400:1700]
y_test_trimmed = y_test[1400:1700]

In [6]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=6,
    input_shape=(33,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [7]:
attack = DeepFool(classifier=classifier, epsilon=0.01)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool:   0%|          | 0/300 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 300/300 [00:11<00:00, 26.26it/s]


In [8]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 1ms/step
Accuracy: 0.24333333333333335
AUC: 0.5518087132274815

macro
Precision: 0.5736281929990539
Recall: 0.21777389277389278
F1 Score: 0.10335622390487936

weighted
Precision: 0.6625999684642069
Recall: 0.24333333333333335
F1 Score: 0.22461601711536905

Mean FNR: 0.7822261072261072
Mean TNR: 0.83114584576224
Mean FPR: 0.16885415423776004
Mean TPR: 0.21777389277389278


In [9]:
attack = DeepFool(classifier=classifier, epsilon=0.05)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [00:11<00:00, 26.25it/s]


In [10]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 888us/step
Accuracy: 0.24
AUC: 0.5543646439679756

macro
Precision: 0.573203201310773
Recall: 0.2164918414918415
F1 Score: 0.10238647430298285

weighted
Precision: 0.6618541450042538
Recall: 0.24
F1 Score: 0.2227127696956919

Mean FNR: 0.7835081585081585
Mean TNR: 0.8309728353816173
Mean FPR: 0.16902716461838285
Mean TPR: 0.2164918414918415


In [9]:
attack = DeepFool(classifier=classifier, epsilon=0.1)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool:   0%|          | 0/300 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 300/300 [04:42<00:00,  1.06it/s]


In [10]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 822us/step
Accuracy: 0.37
AUC: 0.6679295307953943

macro
Precision: 0.1433740250338611
Recall: 0.20640609390609388
F1 Score: 0.15675555892346835

weighted
Precision: 0.29206181151877875
Recall: 0.37
F1 Score: 0.3082362890620278

Mean FNR: 0.7935939060939061
Mean TNR: 0.8547903095418957
Mean FPR: 0.14520969045810428
Mean TPR: 0.20640609390609388


In [11]:
attack = DeepFool(classifier=classifier, epsilon=0.15)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [04:44<00:00,  1.05it/s]


In [12]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 787us/step
Accuracy: 0.37
AUC: 0.666743588342151

macro
Precision: 0.1433740250338611
Recall: 0.20640609390609388
F1 Score: 0.15675555892346835

weighted
Precision: 0.29206181151877875
Recall: 0.37
F1 Score: 0.3082362890620278

Mean FNR: 0.7935939060939061
Mean TNR: 0.8547903095418957
Mean FPR: 0.14520969045810428
Mean TPR: 0.20640609390609388


In [13]:
attack = DeepFool(classifier=classifier, epsilon=0.2)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [04:45<00:00,  1.05it/s]


In [14]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 744us/step
Accuracy: 0.37
AUC: 0.6656662022307355

macro
Precision: 0.14552425407874786
Recall: 0.20640609390609388
F1 Score: 0.15777386094910212

weighted
Precision: 0.2936741205125785
Recall: 0.37
F1 Score: 0.30925605001167705

Mean FNR: 0.7935939060939061
Mean TNR: 0.8551900640420453
Mean FPR: 0.1448099359579547
Mean TPR: 0.20640609390609388


In [15]:
attack = DeepFool(classifier=classifier, epsilon=0.25)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [04:46<00:00,  1.05it/s]


In [16]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 740us/step
Accuracy: 0.37
AUC: 0.6647721301623806

macro
Precision: 0.14329757605222848
Recall: 0.20640609390609388
F1 Score: 0.1566787828552668

weighted
Precision: 0.29332137749986725
Recall: 0.37
F1 Score: 0.3090860540007531

Mean FNR: 0.7935939060939061
Mean TNR: 0.8551940004300157
Mean FPR: 0.1448059995699843
Mean TPR: 0.20640609390609388


In [17]:
attack = DeepFool(classifier=classifier, epsilon=0.3)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [04:47<00:00,  1.04it/s]


In [18]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 771us/step
Accuracy: 0.37
AUC: 0.6631944443196393

macro
Precision: 0.14329757605222848
Recall: 0.20640609390609388
F1 Score: 0.1566787828552668

weighted
Precision: 0.29332137749986725
Recall: 0.37
F1 Score: 0.3090860540007531

Mean FNR: 0.7935939060939061
Mean TNR: 0.8551940004300157
Mean FPR: 0.1448059995699843
Mean TPR: 0.20640609390609388
